# Fetch Forecast for multiple indicators

Welcome to this tutorial! In this notebook, we will explore various methods to retrieve weather data from the ARPEGE and AROME models using a list of indicators. This tutorial aims to provide a comprehensive guide on how to efficiently access data for different scenarios.

We will cover the following cases:
- Retrieving data based on a list of indicators
- For a single run date
- Specifying the "interval" parameter for each indicator
- Specifying the "forecast_horizon" parameter
- Specifying the "heights" or "pressures" parameters for each indicator
- For multiple runs


In [ ]:
import random

from meteole import ArpegeForecast

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("meteole")

**Requirements notice** : TODO Link to the documentation to have application_id

In [ ]:
APP_ID = "<TO-BE-SET>"

## Init Client Arpege

To get Arome Forecast, import `AromeForecast`

In [ ]:
client = ArpegeForecast(application_id=APP_ID)

In [ ]:
# Select random indicators to get
random_indicators = random.sample(client.INDICATORS, 5)
print(f"Selected Indicators: {random_indicators}")

### For One Run

To retrieve data from a list of indicators, it is possible to do so from one or multiple runs common to all indicators. Let's start with the simple case of a single run


In [ ]:
if not hasattr(client, "capabilities"):
    client.get_capabilities()

capabilities = client.capabilities[client.capabilities["indicator"] == random_indicators[0]]
run = capabilities.run.max()

print(run)

In [ ]:
df_combined = client.get_combined_coverage(
    indicator_names=random_indicators,
    runs=[run],
)

In [ ]:
df_combined.head(2)

### Specify intervals

For an indicator, it is sometimes necessary to specify an "interval" parameter. To use the `get_combined_coverage` method, you must provide a list of intervals (`intervals`) associated with the list of indicators (`indicator_names`). If this is not done and an "interval" is required, a default interval will be used, and the default value will be logged as information. If you want to have multiple intervals for the same indicator, you need to duplicate the indicator in `indicator_names`.

In [ ]:
# Depending on the indicators, the parameters intervals must be specified, if needed, we pick the first value
intervals_list = []
for indicator in random_indicators:
    capabilities = client.capabilities[client.capabilities["indicator"] == indicator]
    intervals = capabilities.iloc[0]["interval"]
    intervals_list.append(intervals)

print(f"Intervals: {intervals_list}")

In [ ]:
capabilities.head()

In [ ]:
df_combined = client.get_combined_coverage(
    indicator_names=random_indicators,
    runs=[run],
    intervals=intervals_list,
)

In [ ]:
df_combined.head(2)

### Specify list of forecast_horizons

The list of forecast horizons must be the same and valid for all indicators because the indicators are concatenated into columns for similar forecast horizons. If you want different forecast horizons for different indicators, you need to make multiple calls to `get_combined_coverage`.

In [ ]:
# if you don't know common forecast_horizons between list of indicators, we can pick 2 common forecast_horizons like that
if intervals_list:
    intervals_list = [None if interval == "" else interval for interval in intervals_list]
list_coverage_id = [
    client._get_coverage_id(indicator, run, interval) for indicator, interval in zip(random_indicators, intervals_list)
]
forecast_horizons = client.find_common_forecast_horizons(list_coverage_id)[:2]
print(forecast_horizons)

In [ ]:
df_combined = client.get_combined_coverage(
    indicator_names=random_indicators,
    runs=[run],
    intervals=intervals_list,
    forecast_horizons=forecast_horizons,
)

In [ ]:
df_combined.head(2)

### Specify list of 'heights' or 'pressures'

Depending on the indicators, you need to select `heights` and `pressures` from a list of values. If you do not specify any values, a default value will be used. The list of `heights` or `pressures` must be the same length as `indicator_names`. If you want multiple heights or pressures for the same indicator, you need to duplicate the indicator in `indicator_names`.

In [ ]:
# To get heights, we can use `self.get_coverage_description` and pick a random height if height exist for the indicator
heights = []
for indicator, interval in zip(random_indicators, intervals_list):
    coverage_id = client._get_coverage_id(indicator, run, interval)
    description = client.get_coverage_description(coverage_id)

    # Get a random height if heights exist for the indicator
    possible_heights = description.get("heights", [])
    if possible_heights:
        random_height = random.choice(possible_heights)
        heights.append(random_height)
    else:
        heights.append(None)

print(heights)

In [ ]:
df_combined = client.get_combined_coverage(
    indicator_names=random_indicators,
    runs=[run],
    intervals=intervals_list,
    forecast_horizons=forecast_horizons,
    heights=heights,
)

In [ ]:
df_combined.head(2)

In [ ]:
df_combined.shape

### For multiple runs

If you need to retrieve data for multiple runs, it is possible. Different runs will be concatenated into rows, unlike indicators which are concatenated into columns. Therefore, you need to specify different runs as each run applies to the entire set of indicators.

In [ ]:
capabilities = client.capabilities[client.capabilities["indicator"] == random_indicators[0]]
runs = capabilities["run"].unique()[:2].tolist()

print(runs)

In [ ]:
df_combined = client.get_combined_coverage(
    indicator_names=random_indicators,
    runs=runs,
    intervals=intervals_list,
    forecast_horizons=forecast_horizons,
    heights=heights,
)

In [ ]:
df_combined.head(2)